# Build Your AI Agent with MCPs using SGLang, Pydantic AI, and AMD MI300X GPU

Welcome to this hands-on workshop! In this tutorial, we'll leverage AMD GPUs and **Model Context Protocol (MCP)** ,an open standard for exposing LLM tools via API, to deploy powerful language models like Qwen3-30B. Key components:
- 🖥️ **SGLang** for GPU-optimized inference
- 🛠️ **Pydantic-AI** for agent/tool management
- 🔌 **MCP Servers** for pre-built tool integration

You'll learn how to set up your environment, deploy large language models like Qwen3-30B, connect them to real-world tools using MCP, and build a conversational agent capable of reasoning and taking actions

By the end of this workshop, you’ll have built an AI-powered assistant agent that can find a place to stay based on your preferences like location, budget, and travel dates.

Let’s dive in!

## Table of Contents

- [Step 1: Launching SGLang Server on AMD GPUs](#step1)
- [Step 2: Installing Dependencies](#step2)
- [Step 3: Create a simple instance of Pydantic-AI Agent](#step3)
- [Step 4: Write a Date/Time Tool for your Agent](#step4)
- [Step 5: Replace your Date/Time Tool with a MCP Server](#step5)
- [Step 6: Turn your Agent into a trip planner](#step6)
- [Step 7: Challenge](#step7)

<a id="step1"></a>

## Step 1: Launch SGLang Server on AMD GPUs

In this workshop, you are going to use [SGLang](https://github.com/sgl-project/sglang) inference serving engine. SGLang provides many benefits such as fast model execution, extensive list of supported models, easy to use, and best of all it's open-source. 

### Deploy Qwen3-30B Model with SGLang (~2mins)



First, you need to start a SGLang server and create an OpenAI-compatible endpoint for your LLM:

In [ ]:
import os
from sglang.utils import launch_server_cmd
os.environ["SGLANG_USE_AITER"] = "1"
server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --served-model-name Qwen/Qwen3-30B-A3B-Instruct-2507 --model-path /models/Qwen3-30B-A3B-Instruct-2507 --tool-call-parser qwen25 --host 0.0.0.0"
)

BASE_URL = f"http://localhost:{port}/v1"

# Set environment variables
os.environ["BASE_URL"]    = BASE_URL
os.environ["OPENAI_API_KEY"] = "abc-123"   

print("Config set:", BASE_URL)

***Before continuing, wait until you see "The server is fired up and ready to roll!"***


Once the server is ready, you can verify the model is available with a curl command.

In [ ]:
!curl $BASE_URL/models -H "Authorization: Bearer $OPENAI_API_KEY"

Congratulations, you just launched a powerful server that can serve any incoming request and allowing you to build amazing applications. Wasn't that easy?🎉 

<a id="step2"></a>

## Step 2: Installing Dependencies

Install the dependencies for this notebook using pip:

In [ ]:
!pip install -q pydantic_ai openai  


<a id="step3"></a>

## Step 3: Create a simple instance of Pydantic-AI Agent

Start by creating a custom OpenAI Compatible endpoint for your agent. 


In [ ]:
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider

provider = OpenAIProvider(
    base_url=os.environ["BASE_URL"],
    api_key=os.environ["OPENAI_API_KEY"],
)

agent_model = OpenAIModel("Qwen/Qwen3-30B-A3B-Instruct-2507", provider=provider)

Now create an instance of the `Agent` class from `pydantic_ai`.

In [ ]:
from pydantic_ai import Agent

agent = Agent(
    model=agent_model
)

It's time to test the agent. `pydantic_ai` provides multiple ways to run `Agent`. You can learn more about it from [the PydanticAI site](https://ai.pydantic.dev/agents/#running-agents).

For this workshop, you'll run the agent in async mode to make testing much faster. Let's define the helper function.

In [ ]:
async def run_async(prompt: str) -> str:
    async with agent:
        result = await agent.run(prompt)
        return result.output

Now you can test the agent by calling this function.

In [ ]:
await run_async("What is the capital of France?")

<a id="step4"></a>

## Step 4: Write a Date/Time Tool for Your Agent

LLMs rely on their training data to respond to your prompts, so the agent you just defined will fail to answer factual questions that fall outside of its training knowledge. You can see this with an example:

In [ ]:
await run_async("What’s the date today?")

It is no surprise that the model failed to answer this question. Now, it's time to power-up your LLM by providing `agent` a function that can get the current date. The process of an LLM triggering a function call is commonly referred to as `Tool Calling` or `Function Calling`. In this workshop we are going to take advantage of `pydantic-ai`'s agent `tools` to provide your agent with appropriate tools. First, define a custom tool within this framework using the code sample below.

In [ ]:
from datetime import datetime
from pydantic_ai import Tool          
@Tool
def get_current_date() -> str:
    """Return the current date/time as an ISO-formatted string."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

Next, provide this tool to your agent by adding the function signature of the tool to the `Agent` constructor. This notifies the LLM that the new tool exists.

In [ ]:
agent = Agent(
    model=agent_model,
    tools=[get_current_date],
    system_prompt = (
        "You have access to:\n"
        "   1. get_current_time(params: dict)\n"
        "Use this tool for date/time questions."
    )
)

Now test the agent.

In [ ]:
await run_async("What’s the date today?")

Congratulations on building an agent with access to real-time data.



<a id="step5"></a>

## Step 5: Replace Your Date/time Tool with a MCP Server

Now that you learned how to create a custom tool and let the agent access it, you can enhance this step using [Model Context Protocol](https://modelcontextprotocol.io/introduction). You can replace the custom tool with a simple MCP server that serves the agent and provides similar information.

**Why MCP?** MCP servers provide:
- ✅ Standardized API interfaces
- 🔄 Reusable across projects
- 📦 Pre-built functionality

Let's replace your custom time tool with an official MCP time server.

### Installing Time MCP Server

Start by installing a [Time/Date MCP server](https://github.com/modelcontextprotocol/servers/tree/main/src/time):


In [ ]:
!pip install -q mcp-server-time

Now define the time_server:

In [ ]:
from pydantic_ai.mcp import MCPServerStdio

time_server = MCPServerStdio(
    "python",
    args=[
        "-m", "mcp_server_time",
        "--local-timezone=America/New_York",
    ],
)

Finally, modify your agent by removing the previously defined tool and adding the MCP server instead.

In [ ]:
agent = Agent(
    model=agent_model,
    toolsets=[time_server],
    system_prompt = (
        "You are a helpful agent and you have access to this tool:\n"
        "   get_current_time(params: dict)\n"
        "When the user asks for the current date or time, call get_current_time.\n"
    )
)

Great, you can now see whether the agent can use MCP to provide the correct time.

In [ ]:
await run_async("What’s the date today?")


Tadaa! Now you have officially used MCP servers to power-up your AI agents. In the next section we show how you can your turn many ideas into real working projects by using free MCP servers available today.



<a id="step6"></a>

## Step 6: Turn your agent into a travel planner

As you saw in the last section, MCP servers are really easy to use and they provide a standard way of providing LLMs the tools they need. There are already thousands of MCP servers available to use and MCP trackers that share available servers. Here are some for your reference:
- https://github.com/modelcontextprotocol/servers
- https://mcp.so/

You are going to use npx to launch out the next server. Let's install the required dependencies first.

In [ ]:
# Install Node.js 20 via NodeSource
!curl -fsSL https://deb.nodesource.com/setup_20.x | sudo -E bash -
!apt install -y nodejs

Verify `npm` and `npx` installation:

In [ ]:
!node -v && npm -v && npx --version

In this part of the workshop we are going to build an agent that can help you browse available Airbnbs to book. We can now build on top of what we have so far and add an open-source [Airbnb MCP server](https://github.com/openbnb-org/mcp-server-airbnb) to our agent. To do so, let's start by defining our Airbnb server.

In [ ]:
airbnb_server = MCPServerStdio(
    "npx", args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"]
)

Let's update our agent.

In [ ]:
system_prompt = """
You have access to three tools:
1. get_current_time(params: dict)
2. airbnb_search(params: dict)
3. airbnb_listing_details(params: dict)
When the user asks for listings, first call get_current_time, then airbnb_search, etc.
"""

agent = Agent(
    model=agent_model,
    toolsets=[time_server, airbnb_server],
    system_prompt=system_prompt,
)

Finally, let's try our agent and see if it can browse through Airbnb listings.

In [ ]:
await run_async("Find a place to stay in San Francisco for next Sunday for 3 nights for 2 adults?")

Happy coding! If you encounter issues or have questions, don’t hesitate to ask or raise an issue on our [Github page](https://github.com/ROCm/gpuaidev)!

<a id="step7"></a>

## Step 7: Challenge - Expand the Agent

**Task:** The challenge for this workshop will be announced during the workshop. 

You can open a terminal and watch the GPU utilization by running this command:

```bash 
watch rocm-smi
```

Let's set some environment variables for our server to use throughout this tutorial:

# Clean up the process of SGLang server

In [ ]:
from sglang.utils import terminate_process
terminate_process(server_process)